In [17]:
import pandas as pd
import numpy as np
import statistics 

users_df = pd.read_csv("E:/ASU_CourseWork/Fall_2018/SML/Project/sof_user_churn/data/processed/users.csv", delimiter = ',')
posts_df = pd.read_csv("E:/ASU_CourseWork/Fall_2018/SML/Project/sof_user_churn/data/processed/posts.csv", delimiter = ',')
posts_df.head()

,Id,OwnerUserId,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,LastEditorUserId,LastEditDate,LastActivityDate,CommentCount
0,5,5.0,1,NaN,NaN,2014-05-13T23:58:30.457,9,NaN,NaN,2014-05-14T00:36:31.077,1
1,7,36.0,1,10.0,NaN,2014-05-14T00:11:06.457,4,97.0,2014-05-16T13:45:00.237,2014-05-16T13:45:00.237,4
2,9,51.0,2,NaN,5.0,2014-05-14T00:36:31.077,5,NaN,NaN,2014-05-14T00:36:31.077,0
3,10,22.0,2,NaN,7.0,2014-05-14T00:53:43.273,12,NaN,NaN,2014-05-14T00:53:43.273,1
4,14,66.0,1,29.0,NaN,2014-05-14T01:25:59.677,21,322.0,2014-06-17T16:17:20.473,2014-06-20T17:36:05.023,1


In [34]:
accepted_answerer_data = []
userId_list = users_df['Id']
for user in userId_list:
    accepted_postid_list = posts_df[(posts_df.OwnerUserId == user) & (posts_df.PostTypeId == 1) & 
                                   (posts_df.AcceptedAnswerId.notnull())]['AcceptedAnswerId']
    accepted_answerer_userIds = posts_df[posts_df.Id.isin(accepted_postid_list)]['OwnerUserId']
    mean_rep = users_df[users_df.Id.isin(accepted_answerer_userIds)].Reputation.mean()
    accepted_answerer_data.append({'userid' : user, 'mean_reputation' : mean_rep})
accepted_answerer_rep = pd.DataFrame(accepted_answerer_data)


,mean_reputation,userid
0,NaN,-1
1,NaN,1
2,NaN,2
3,NaN,3
4,NaN,4


In [36]:
accepted_answerer_rep[accepted_answerer_rep.mean_reputation.notnull()].head()            

,mean_reputation,userid
11,1109.000000,11
25,2055.333333,26
35,1342.000000,36
60,3874.000000,63
63,374.000000,66


In [9]:
meanOfmax_answerer_reputation_data = []
userId_list = users_df['Id']
for user in userId_list:
    user_question_postid_list = posts_df[(posts_df.OwnerUserId == user) & (posts_df.PostTypeId == 1)]['Id']
    max_rep_list = []
    for postid in user_question_postid_list:
        answerers_userid = posts_df[posts_df.ParentId == postid]['OwnerUserId']
        rept = users_df[users_df.Id.isin(answerers_userid)].Reputation.max()
        max_rep_list.append(rept)
    if (len(max_rep_list) > 0):
        meanOfmax_answerer_reputation_data.append({'userid' : user, 'max_rep_answerer' : np.mean(max_rep_list)})
    
meanOfMax_reputation_answerer = pd.DataFrame(meanOfmax_answerer_reputation_data)
print(meanOfMax_reputation_answerer)

c:\users\ravit\appdata\local\programs\python\python37-32\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\ravit\appdata\local\programs\python\python37-32\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
meanOfMax_reputation_answerer.tail()

,max_rep_answerer,userid
46184,NaN,58496
46185,NaN,58497
46186,NaN,58498
46187,NaN,58499
46188,NaN,58500


In [4]:
userId_to_noofHis_questions_answered = []
userId_list = users_df['Id']
for user in userId_list:
    user_question_post_id_list = posts_df[(posts_df.OwnerUserId == user) & (posts_df.PostTypeId == 1)]['Id']
    user_questions_answered = 0
    for post_id in user_question_post_id_list:
        counter = len(posts_df[posts_df.ParentId == post_id])
        if (counter > 0):
            user_questions_answered += 1
    if (user_questions_answered > 0):
        userId_to_noofHis_questions_answered.append({'userid': user, 'number_ofHis_questions_answered': user_questions_answered})
userId_to_his_questions_answered = pd.DataFrame(userId_to_noofHis_questions_answered)
print(userId_to_his_questions_answered)

      number_ofHis_questions_answered  userid
0                                   1       5
1                                   1      11
2                                   5      26
3                                   1      34
4                                   2      36
5                                   1      62
6                                   1      63
7                                   1      66
8                                   1      71
9                                   1      75
10                                  1      77
11                                  3      82
12                                 12      84
13                                  5      88
14                                  2      95
15                                  1      96
16                                 14      97
17                                  1     102
18                                  1     109
19                                  2     113
20                                

In [20]:
from datetime import datetime
import time

fmt = '%Y-%m-%d %H:%M:%S'

userId_to_mean_time_for_first_answ = []
userId_list = users_df['Id']
for user in userId_list:
    #user_question_post_id_df
    df = posts_df[(posts_df.OwnerUserId == user) & (posts_df.PostTypeId == 1)][['Id', 'CreationDate']]
    first_answered_time_list = []

    for index, row in df.iterrows():
        # Formating the date format of the question created date
        question_date = row['CreationDate']
        question_date = question_date.replace("T", " ")
        question_date = question_date[: len(question_date) - 4]
        d1 = datetime.strptime(question_date, fmt)
        d1_ts = time.mktime(d1.timetuple())
        
        answered_date_list = posts_df[posts_df.ParentId == row['Id']]['CreationDate'].tolist()
        answered_time_diff_list = []
        
        #Formating the date format of the answer created date for the given quesiton and Convert to Unix timestamp
        for date in answered_date_list:
            date = date.replace("T", " ")
            date = date[: len(date) - 4]
            d2 = datetime.strptime(date, fmt)
            d2_ts = time.mktime(d2.timetuple())
            answered_time_diff_list.append(int(d2_ts-d1_ts) / 60)
        
        answered_time_diff_list.sort()
        if (len(answered_time_diff_list) > 0):
            first_answered_time_list.append(answered_time_diff_list[0])

    if (len(first_answered_time_list) > 0):
        mean_response_time = sum(first_answered_time_list)/len(first_answered_time_list)
        userId_to_mean_time_for_first_answ.append({'userid': user, 'time_for_first_answer': mean_response_time})

userId_to_mean_time_for_first_answ_DF = pd.DataFrame(userId_to_mean_time_for_first_answ)
print(userId_to_mean_time_for_first_answ_DF)

      time_for_first_answer  userid
0              3.801667e+01       5
1              2.963333e+02      11
2              9.211133e+04      26
3              2.427667e+02      34
4              4.510475e+03      36
5              1.351685e+06      62
6              6.345500e+02      63
7              3.905833e+02      66
8              2.700133e+03      71
9              4.352167e+02      75
10             6.526999e+05      77
11             7.574889e+02      82
12             1.869448e+05      84
13             8.881333e+01      88
14             4.629358e+04      95
15             1.207500e+02      96
16             3.026202e+03      97
17             8.416667e+01     102
18             1.211333e+02     109
19             1.348358e+03     113
20             2.659167e+02     116
21             8.425500e+02     118
22             2.125614e+04     122
23             2.427167e+02     131
24             6.688782e+03     133
25             3.655000e+01     134
26             1.892958e+03 